In [143]:
%run Model.ipynb

In [144]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pytorch_lightning as pl
from torch.utils.data import DataLoader

In [145]:
class MnistData(pl.LightningDataModule):
    def __init__(self, batch_size, num_workers, pin_memory):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.transform = transforms.Compose([
            transforms.Resize((image_size,image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5 for _ in range(channels_img)],[0.5 for _ in range(channels_img)]),])
        
    def prepare_data(self):
        mnist_train = datasets.MNIST('./data', train=True, download=True,)
        mnist_test = datasets.MNIST('./data', train=False, download=True,)

    def setup(self, stage=None):
        self.train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=self.transform)
        self.test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=self.transform)
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=self.pin_memory)
    
    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=self.pin_memory)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=self.pin_memory)
    

In [146]:
z_dim = 100
batch_size = 128
num_epochs = 5
channels_img = 1
image_size = 64
featurees_disc = 64
feature_gen =64

In [149]:
dm = MnistData(batch_size, num_workers=4, pin_memory=True)

In [153]:
trainer = pl.Trainer(accelerator = "auto",devices=1 if torch.cuda.is_available() else None, max_epochs=num_epochs)
trainer.fit(Modelpt(z_dim, channels_img, feature_gen, featurees_disc, batch_size), dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type          | Params
-------------------------------------------
0 | D        | Discriminator | 2.8 M 
1 | G        | Generator     | 12.7 M
2 | BCE_loss | BCELoss       | 0     
-------------------------------------------
15.4 M    Trainable params
0         Non-trainable params
15.4 M    Total params
61.672    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]